# **Pan-sharpening**

เทคนิค Pan-sharpening เป็นเทคนิคที่ใช้กับภาพถ่ายดาวเทียม ที่มี Panchromatic band เช่น THEOS-1, LANDSAT-8, LANDSAT-9 เพื่อให้มีรายละเอียดและมีประโยชน์มากขึ้น โดย Lab นี้เราจะทดลองกับข้อมูลดาวเทียม Landsat-9

ข้อมูลถ่ายที่เป็น Multi-Spectral มักจะมีรายละเอียดภาพไม่ค่อยดีนัก เมื่อเทียบกับข้อมูล Panchromatics  ดังนั้น การปรับความคมชัดของภาพช่วยได้โดยการรวมภาพเหล่านี้เข้ากับภาพขาวดำพิเศษที่มีรายละเอียดมากขึ้น ภาพขาวดำที่เรียกว่าภาพแพนโครมาติก มีรายละเอียดในระดับที่สูงกว่า แต่ไม่มีสี เราต้องการใช้รายละเอียดจากภาพแพนโครมาติกและสีจากภาพ Landsat-9 เพื่อสร้างภาพสีที่มีรายละเอียดสูงใหม่

ในการดำเนินการใน Lab นี้ นักศึกษา จำเป็นต้องตรวจสอบให้แน่ใจว่าข้อมูลทั้ง Panchromatic และ Multispectral ของ Landsat-9 อยู่ในตำแหน่งที่สมบูรณ์แบบ เพื่อให้เข้ากันได้อย่างลงตัว

มีหลายวิธีในการรวมภาพเหล่านี้ เช่น Brovey Transform, IHS Transform และ PCA วิธีการเหล่านี้ทำให้แน่ใจว่าภาพใหม่จะคงสีจาก Landsat-9 แต่ยังได้รับรายละเอียดเพิ่มเติมจากภาพแพนโครมาติกด้วย


เมื่อเราทำ Pan-sharpening เสร็จแล้ว เราก็จะได้ภาพใหม่ที่เป็นภาพสีที่มีรายละเอียดสูงขึ้น  โดยมีสีทั้งหมดจาก Landsat-9 และความคมชัดพิเศษจากภาพแบบแพนโครมาติก รูปภาพใหม่นี้สามารถนำไปใช้ได้หลายอย่าง เช่น ศึกษาพื้นดิน ค้นหาการเปลี่ยนแปลง หรือเพียงแค่ดูรายละเอียดเพิ่มเติมเกี่ยวกับโลก ดังนั้น การปรับความคมชัดของภาพเป็นวิธีหนึ่งในการทำให้ภาพจากดาวเทียมมีประโยชน์มากขึ้นสำหรับงานสำคัญทุกประเภท

In [ ]:
# ทำการ Authenticate และ initialize Earth Engine
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='ee-suranajkrua165') #อย่าลืมเปลี่ยนชื่อโปรเจคของตัวเอง

In [ ]:
# กำหนดพื้นที่สนใจ
geometry = ee.Geometry.Point([98.95799098999555, 18.84423947416328])

# เรียกภาพ L9 ตัวอย่างแล้วดึง RGB และ Pan ออกมา
image = (ee.ImageCollection("LANDSAT/LC09/C02/T1_TOA")
         .filterDate('2022-01-01', '2022-03-30')
         .filterBounds(geometry)
         .sort('CLOUD_COVER')
         .first())

In [ ]:
# ทำการ Pan-Sharp
rgb = image.select('B4', 'B3', 'B2')
pan = image.select('B8')

# แปลงเป็น HSV, สลับในแถบ PAN และแปลงกลับเป็น RGB
huesat = rgb.rgbToHsv().select('hue', 'saturation')
upres = ee.Image.cat([huesat, pan]).hsvToRgb()

In [ ]:
# สร้างแผนที่
Map = geemap.Map(center=[18.84423947416328, 98.95799098999555], zoom=14)

# แสดง เลเยอร์ ก่อนและหลังโดยใช้พารามิเตอร์ vis เดียวกัน
Map.addLayer(rgb, {'max': 0.28}, 'Original')
Map.addLayer(pan, {'max': 0.28}, 'Pan')
Map.addLayer(upres, {'max': 0.28}, 'Pansharpened')
Map


Map(center=[18.84423947416328, 98.95799098999555], controls=(WidgetControl(options=['position', 'transparent_b…

คำถาม
ข้อเพื่อทดสอบความเข้าใจของนักศึกษาเกี่ยวกับเทคนิคการทำ Pan-sharpening ด้วยภาพ Landsat-9 จงตอบคำถามต่อไปนี้

1. อะไรคือเป้าหมายหลักของการปรับความคมชัดของภาพในการสำรวจระยะไกล โดยเฉพาะเมื่อใช้ภาพ Landsat-9 อธิบายว่าเหตุใดจึงมีความสำคัญในการประมวลผลภาพ

2. อธิบายขั้นตอนสำคัญที่เกี่ยวข้องกับการปรับความคมชัดด้วนเทคนิค Pan-sharpening ตั้งแต่การรับข้อมูลไปจนถึงการสร้างภาพที่ปรับความคมชัด เทคนิค Pan-sharpening มี กระบวนการสุ่มตัวอย่างใหม่ช่วยจัดแนวภาพหลายสเปกตรัมและภาพแพนโครมาติกอย่างไร


ข้อ1. ตอบ เป้าหมายหลักของการปรับความคมชัดของภาพในการสํารวจระยะไกล โดยเฉพาะเมื่อใช้ภาพ Landsat-9 ด้วยเทคนิค Pan-sharpening คือ การทําให้ภาพมีความละเอียดเเละคมชัดมากยิ่งขึ้น กระบวนการนี้เป็นการนํา Panchromatic Band ซึ่งเป็นภาพขาวดํา ที่มีความละเอียดจุดภาพสูง 15 เมตร มาเป็นตัวตั้งเเล้วนําข้อมูลภาพสีจาก Multispectral Band ซึ่งมีความละเอียด 30 เมตร มารวมเข้าด้วยกัน ผลลัพธ์ที่ได้ คือ ภาพสีที่มีความละเอียด 15 เมตร เท่ากับเเบด์ panchromatic ซึ่งช่วยเเก้ปัญหาภาพเบลอจากการใช้เพียงเเบนด์สีปกติ

การทํา Pan-sharpening มีความสําคัญอย่างมากในงานด้านการทําเเผนที่เเละการเเปลความด้วยสายตา  
ยกตัวอย่างเช่น
*   ช่วยให้มองเห็นรายละเอียดของวัตถุได้ชัดเจนขึ้น เช่น การเปรียบเทียบภาพบริเวณโรงงานหรือหมู่บ้านจัดสรร ซึ่งภาพปกติ 30 เมตร จะดูเบลอ เเต่เมื่อทํา Pan sharpening จะได้ภาพใหม่ที่เป็นภาพสีที่มีรายละเอียดสูงขึ้น จะเห็นขอบเขตของอาคาร ถนน  เเละเส้นทางน้ำได้อย่างชัดเจน เพิ่มความถูกต้องของขอบเขตพื้นที่

*   เทคนิคนี้ช่วยให้ได้ภาพที่สวยงามและคมชัด เหมาะสำหรับการนำไปซ้อนทับข้อมูล GIS หรือทำแผนที่ฐาน (Basemap) ที่เน้นการดูเเปลตา เหมาะกับการนําภาพไปสร้าง Shape file กั้นอาณาเขต

ข้อจํากัดที่สำคัญ แม้ภาพจะคมชัดขึ้น แต่ ไม่ควรนำภาพที่ผ่านการทำ Pan Sharpening ไปใช้ในการวิเคราะห์เชิงรังสี (Spectral Analysis) หรือการจำแนกวัตถุอัตโนมัติ (Classification) เพราะ กระบวนการนี้ทำให้ค่าการสะท้อนแสง (Digital Number) ผิดเพี้ยนไปจากค่าจริงที่เซนเซอร์บันทึกได้

ข้อ2. ตอบ ขั้นตอนสำคัญของการทำ Pan-sharpening ตั้งแต่เริ่มต้นจนถึงได้ภาพผลลัพธ์ รวมถึงบทบาทของการสุ่มตัวอย่างใหม่ (Resampling) ในการจัดแนวภาพ มีรายละเอียดดังนี้
1. ขั้นตอนสำคัญของ Pan-sharpening
กระบวนการนี้มีเป้าหมายเพื่อสร้างภาพสีที่มีความละเอียดสูง โดยการรวมข้อมูลจากภาพมัลติสเปกตรัม (สี) และภาพแพนโครมาติก (ขาว-ดำละเอียดสูง) เข้าด้วยกัน ผ่านขั้นตอนดังนี้

• ขั้นตอนที่ 1: การรับและเตรียมข้อมูล (Data Acquisition & Preparation) เริ่มต้นด้วยการเชื่อมต่อกับ Earth Engine และดึงข้อมูลภาพดาวเทียม (ในที่นี้คือ Landsat-9) โดยกำหนดพื้นที่และช่วงเวลาที่ต้องการ พร้อมทั้งคัดกรองเมฆหรือทำ Cloud Masking เพื่อให้ได้ภาพที่สะอาดและมีคุณภาพดีที่สุด

• ขั้นตอนที่ 2: การเลือกแบนด์ (Band Selection) ระบบจะแยกข้อมูลออกเป็น 2 ส่วน
    1. RGB Bands เลือกแบนด์สีธรรมชาติ (Red, Green, Blue) คือ แบนด์ 4, 3, 2 ซึ่งมีความละเอียด 30 เมตร
    2. Panchromatic Band เลือกแบนด์ 8 ซึ่งเป็นภาพขาว-ดำที่มีความละเอียดสูง 15 เมตร

• ขั้นตอนที่ 3 การแปลงระบบสี (RGB to HSV Transformation) นำภาพ RGB (30 เมตร) มาแปลงระบบสีไปเป็น HSV (Hue, Saturation, Value) เพื่อแยก "ค่าสี" (Hue/Saturation) ออกจาก "ค่าความสว่าง" (Value)

• ขั้นตอนที่ 4 การแทนที่และหลอมรวม (Substitution/Fusion) นำข้อมูลจาก Panchromatic Band (15 เมตร) เข้ามาแทนที่ในส่วนของ Value (ค่าความสว่าง) ของระบบ HSV เดิม วิธีนี้เป็นการนำรายละเอียดความคมชัดสูงของภาพขาว-ดำมาใส่แทนความสว่างเดิมของภาพสี

• ขั้นตอนที่ 5 การแปลงกลับเป็น RGB (HSV to RGB) แปลงข้อมูลจากระบบ HSV (ที่ถูกแทนที่ค่าความสว่างแล้ว) กลับมาเป็นระบบ RGB ผลลัพธ์ที่ได้คือภาพสีที่มีโครงสร้างความคมชัดระดับ 15 เมตร

2. กระบวนการสุ่มตัวอย่างใหม่ (Resampling) ช่วยจัดแนวภาพอย่างไร?

ในทางเทคนิคของการประมวลผลภาพ การจัดแนวภาพให้ซ้อนทับกันสนิท (Co-registration) ระหว่างสองความละเอียดเป็นสิ่งสำคัญมาก

• ใช้ภาพ Pan เป็นโครงสร้างหลัก (Reference Grid): กระบวนการนี้จะใช้ตารางกริด (Raster Grid) ของภาพ Panchromatic ซึ่งมีความละเอียด 15 เมตร เป็น "ตัวตั้ง" หรือแม่แบบ

• การสุ่มตัวอย่างใหม่เพื่อจัดแนว (Resampling for Alignment)
ข้อมูลสีจากภาพมัลติสเปกตรัม (30 เมตร) จะถูกสุ่มตัวอย่างใหม่ (Resampled) เพื่อให้ขนาดพิกเซลลดลงจาก 30 เมตร มาเท่ากับ 15 เมตร เพื่อให้สอดคล้องกับกริดของภาพ Pan การทำเช่นนี้ช่วยให้มั่นใจว่าค่าสี (Hue/Saturation) จะถูกเกลี่ยลงในตำแหน่งที่ถูกต้องตรงกับรายละเอียดโครงสร้างของภาพ Pan อย่างสมบูรณ์



